In [1]:
import import_ipynb #allows access to import other notebooks
from PreprocessLib import cleanText, removeSingleLists, process_file

importing Jupyter notebook from PreprocessLib.ipynb


In [2]:
#processes files for Word2Vec
clemFile = open("./finalTexts/clement.txt", "r", encoding="utf8")
lukeFile = open("./finalTexts/luke.txt", "r", encoding="utf8")
pastFile = open("./finalTexts/pastoral.txt", "r", encoding="utf8")
paulFile = open("./finalTexts/paul.txt", "r", encoding="utf8")
testFile = open("./finalTexts/hebrews.txt", "r", encoding="utf8")
clem = process_file(clemFile)
luke = process_file(lukeFile)
past = process_file(pastFile)
paul = process_file(paulFile)
test = process_file(testFile)
#processes files for POS tagging
clemFile = open("./finalTexts/clement.txt", "r", encoding="utf8")
lukeFile = open("./finalTexts/luke.txt", "r", encoding="utf8")
pastFile = open("./finalTexts/pastoral.txt", "r", encoding="utf8")
paulFile = open("./finalTexts/paul.txt", "r", encoding="utf8")
testFile = open("./finalTexts/hebrews.txt", "r", encoding="utf8")
clemFreq = process_file(clemFile, False)
lukeFreq = process_file(lukeFile, False)
pastFreq = process_file(pastFile, False)
paulFreq = process_file(paulFile, False)
testFreq = process_file(testFile, False)

In [27]:
import gensim
from gensim.models import Word2Vec  
#best model
clemModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=100, sg=1) 
clemModel.build_vocab(clem)
clemModel.train(clem, total_examples=clemModel.corpus_count, epochs=clemModel.iter)

lukeModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=100, sg=1) 
lukeModel.build_vocab(luke)
lukeModel.train(luke, total_examples=lukeModel.corpus_count, epochs=lukeModel.iter)

pastModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=100, sg=1) 
pastModel.build_vocab(past)
pastModel.train(past, total_examples=pastModel.corpus_count, epochs=pastModel.iter)

paulModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=100, sg=1) 
paulModel.build_vocab(paul)
paulModel.train(paul, total_examples=paulModel.corpus_count, epochs=paulModel.iter)

testModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=100, sg=1) 
testModel.build_vocab(test)
testModel.train(test, total_examples=testModel.corpus_count, epochs=testModel.iter)

<ipython-input-27-ab074bc1e01f>:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  clemModel.train(clem, total_examples=clemModel.corpus_count, epochs=clemModel.iter)
<ipython-input-27-ab074bc1e01f>:10: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  lukeModel.train(luke, total_examples=lukeModel.corpus_count, epochs=lukeModel.iter)
<ipython-input-27-ab074bc1e01f>:14: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  pastModel.train(past, total_examples=pastModel.corpus_count, epochs=pastModel.iter)
<ipython-input-27-ab074bc1e01f>:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  paulModel.train(paul, total_examples=paulModel.corpus_count, epochs=paulModel.iter)
<ipython-input-27-ab074bc1e01f>:22: DeprecationWarning: Call to deprecated `i

(11183, 20000)

In [28]:
#takes in a W2V model and returns the associated vocabulary of the model
def intoDict (model): 
    # Get the ordered list of words in the vocabulary
    words = model.wv.vocab.keys()
    # Make a dictionary
    we_dict = {word:model.wv[word] for word in words}
    return we_dict

In [29]:
#takes a dictionary "dct", a list of words "wordList", and optionally a label
#takes each word vector of the words in wordList and adds them to an overall vector, then returns this vector
def createVec(dct, wordList, label="NA"):
    vec = []
    for word in wordList:
        vec.append(dct[word])
    if (label != "NA"):
        vec.append(label)
    return vec

In [30]:
#creating feature vectors of the models
clemDict = intoDict(clemModel)
lukeDict = intoDict(lukeModel)
pastDict = intoDict(pastModel)
paulDict = intoDict(paulModel)
testDict = intoDict(testModel)

commonWordList = ['θεοῦ', 'ἡμῶν', 'τὰς', 'ἵνα', 'αὐτοῦ', 'σου', 'κύριος', 'τοῦτο']

clemVec = createVec(clemDict, commonWordList, 'Clement')
lukeVec = createVec(lukeDict, commonWordList, 'Luke')
pastVec = createVec(pastDict, commonWordList, 'Pastoral')
paulVec = createVec(paulDict, commonWordList, 'Paul')
testVec = createVec(testDict, commonWordList)

In [31]:
import numpy as np
#converting to training and testing sets
trainX = np.array((clemVec[:5], lukeVec[:5], pastVec[:5], paulVec[:5]))
trainY =["Clement", "Luke", "Pastoral", "Paul"]
testX = np.array(testVec[:5])

In [32]:
#reshaping the vectors
nsamples, nx, ny = trainX.shape
trainXReshape = trainX.reshape((nsamples,nx*ny))

In [33]:
#MLPClassifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=4000, hidden_layer_sizes=(4,)).fit(trainXReshape, trainY)

In [34]:
#results of MLP classif
testXReshape = testX.reshape(1,testX.shape[1]*5)
print(clf.classes_)
clf.predict_proba(testXReshape)


['Clement' 'Luke' 'Pastoral' 'Paul']


array([[0.07261948, 0.01284027, 0.89705575, 0.01748449]], dtype=float32)

In [35]:
#saving the models
clemModel.save("clemModel.model")
lukeModel.save("lukeModel.model")
pastModel.save("pastModel.model")
paulModel.save("paulModel.model")
testModel.save("testModel.model")

['Clement' 'Luke' 'Pastoral' 'Paul']
[[0.07261948 0.01284027 0.89705575 0.01748449]]


In [75]:
#min_count = 2, alpha=0.035, sample=0.0001, size=5, sg=1
testXReshape = testX.reshape(1,testX.shape[1]*5)
print(clf.classes_)
clf.predict_proba(testXReshape)

['Clement' 'Luke' 'Pastoral' 'Paul']


array([[0.03176393, 0.01781514, 0.9333607 , 0.01706028]], dtype=float32)

In [91]:
#min_count = 2, alpha=0.025, sample=0.0001, size=100, sg=1
testXReshape = testX.reshape(1,testX.shape[1]*5)
print(clf.classes_)
clf.predict_proba(testXReshape)

['Clement' 'Luke' 'Pastoral' 'Paul']


array([[0.12112716, 0.00412895, 0.8714757 , 0.00326816]], dtype=float32)

In [92]:
import numpy as np
#returns cosine similarity of vector A and vector B
def cosineSim(A, B):
    sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    if np.isnan(np.sum(sim)):
        return 0
    return sim

In [142]:
import gensim
from gensim.models import Word2Vec  
clemModel = Word2Vec(min_count = 1, alpha=0.035, sample=0.0001, size=100, sg=1) 
clemModel.build_vocab(clem)
clemModel.train(clem, total_examples=clemModel.corpus_count, epochs=clemModel.iter)

lukeModel = Word2Vec(min_count = 1, alpha=0.035, sample=0.0001, size=100, sg=1) 
lukeModel.build_vocab(luke)
lukeModel.train(luke, total_examples=lukeModel.corpus_count, epochs=lukeModel.iter)

pastModel = Word2Vec(min_count = 1, alpha=0.035, sample=0.0001, size=100, sg=1) 
pastModel.build_vocab(past)
pastModel.train(past, total_examples=pastModel.corpus_count, epochs=pastModel.iter)

paulModel = Word2Vec(min_count = 1, alpha=0.035, sample=0.0001, size=100, sg=1) 
paulModel.build_vocab(paul)
paulModel.train(paul, total_examples=paulModel.corpus_count, epochs=paulModel.iter)

testModel = Word2Vec(min_count = 1, alpha=0.035, sample=0.0001, size=100, sg=1) 
testModel.build_vocab(test)
testModel.train(test, total_examples=testModel.corpus_count, epochs=testModel.iter)

(3110, 20000)

In [143]:
#takes in countNum, which is the minimum threshold for how many documents the word is found in
#this function tests two things
#1) the mean average cosine similarity of the similar words to the test document
#2) a count of which author's words are most similar to the test document's words
def runAnalysis(countNum = 1):
        tDistList = []
        ClDistList = []
        LuDistList = []
        PaDistList = []
        PuDistList = []


        countList = []

        for x in testModel.wv.vocab.keys():
            label = "notinany"
            maxNum = -9999

            count = 0

            if (x in testModel.wv.vocab):
                tDist = cosineSim(testModel[x], testModel[x])
                tDistList.append(tDist)
                
            if (x in clemModel.wv.vocab):
                ClDist = cosineSim(testModel[x], clemModel[x])
                ClDistList.append(ClDist)
                count += 1
                if (ClDist > maxNum):
                    maxNum = ClDist
                    label = "Clement"

            if (x in lukeModel.wv.vocab):
                LuDist = cosineSim(testModel[x], lukeModel[x]) 
                LuDistList.append(LuDist)
                count += 1
                if (LuDist > maxNum):
                    maxNum = LuDist
                    label = "Luke"

            if (x in pastModel.wv.vocab):
                PaDist = cosineSim(testModel[x], pastModel[x])
                PaDistList.append(PaDist)
                count += 1
                if (PaDist > maxNum):
                    maxNum = PaDist
                    label = "Pastoral"

            if (x in paulModel.wv.vocab):   
                PuDist = cosineSim(testModel[x], paulModel[x])
                PuDistList.append(PuDist)
                count += 1
                if (PuDist > maxNum):
                    maxNum = PuDist
                    label = "Paul"


            if (label != 'notinany' and count > countNum):
                countList.append(label)

        
        
        import statistics
        #mean of all cosine similarities of the word to the test word in the list
        print("Test:", statistics.mean(tDistList), "Clement:", statistics.mean(ClDistList), "Luke:", statistics.mean(LuDistList), 
              "Pastoral:", statistics.mean(PaDistList), "Paul:", statistics.mean(PuDistList))
        from collections import Counter
        from collections import OrderedDict
        list = Counter(countList)
        #percentage of words closest to the test word in each list
        print(OrderedDict([(i, str(round(count / sum(list.values()) * 100.0, 3)) + '%') for i, count in list.most_common()]))

In [144]:
import warnings
warnings.filterwarnings('ignore')
#min_count = 2, alpha=0.035, sample=0.0001, size=200, sg=1
runAnalysis(3)
runAnalysis(2)
runAnalysis(1)

Test: 1.0 Clement: 0.7853447 Luke: 0.105903685 Pastoral: 0.9840193 Paul: 0.11034792
OrderedDict([('Pastoral', '100.0%')])
Test: 1.0 Clement: 0.7853447 Luke: 0.105903685 Pastoral: 0.9840193 Paul: 0.11034792
OrderedDict([('Pastoral', '62.745%'), ('Clement', '37.255%')])
Test: 1.0 Clement: 0.7853447 Luke: 0.105903685 Pastoral: 0.9840193 Paul: 0.11034792
OrderedDict([('Pastoral', '45.515%'), ('Clement', '33.887%'), ('Paul', '10.299%'), ('Luke', '10.299%')])


In [141]:
#min_count = 4, alpha=0.025, sample=0.0001, size=30, sg=1
runAnalysis(3)
runAnalysis(2)
runAnalysis(1)

Test: 1.0 Clement: 0.99468833 Luke: 0.099074885 Pastoral: 0.9991536 Paul: 0.21117012
OrderedDict([('Pastoral', '100.0%')])
Test: 1.0 Clement: 0.99468833 Luke: 0.099074885 Pastoral: 0.9991536 Paul: 0.21117012
OrderedDict([('Pastoral', '63.889%'), ('Clement', '36.111%')])
Test: 1.0 Clement: 0.99468833 Luke: 0.099074885 Pastoral: 0.9991536 Paul: 0.21117012
OrderedDict([('Pastoral', '43.81%'), ('Clement', '28.571%'), ('Paul', '20.0%'), ('Luke', '7.619%')])


In [3]:
#POS tags
from angel import tag
clemTags = tag(clemFreq)
lukeTags = tag(lukeFreq)
paulTags = tag(paulFreq)
pastTags = tag(pastFreq)
testTags = tag(testFreq)

Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 8439 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 30454 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood m

In [4]:
#takes in a tag list and transforms it to 3D vector format
def prepareForW2V(tagList):
    cleaned = [[]]
    line = []
    for word in tagList:
        if (word[0] == "."):
            cleaned.append(line)
            line = []
        else:
            line.append(word[1])
    return cleaned

In [6]:
#transforms the texts into vector format
clemPrep = prepareForW2V(clemTags)
lukePrep = prepareForW2V(lukeTags)
paulPrep = prepareForW2V(paulTags)
pastPrep = prepareForW2V(pastTags)
testPrep = prepareForW2V(testTags)

In [8]:
import gensim
from gensim.models import Word2Vec  

clemModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=5, sg=1) 
clemModel.build_vocab(clemPrep)
clemModel.train(clemPrep, total_examples=clemModel.corpus_count, epochs=clemModel.iter)

lukeModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=5, sg=1) 
lukeModel.build_vocab(lukePrep)
lukeModel.train(lukePrep, total_examples=lukeModel.corpus_count, epochs=lukeModel.iter)

pastModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=5, sg=1) 
pastModel.build_vocab(pastPrep)
pastModel.train(pastPrep, total_examples=pastModel.corpus_count, epochs=pastModel.iter)

paulModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=5, sg=1) 
paulModel.build_vocab(paulPrep)
paulModel.train(paulPrep, total_examples=paulModel.corpus_count, epochs=paulModel.iter)

testModel = Word2Vec(min_count = 1, alpha=0.025, sample=0.0001, size=5, sg=1) 
testModel.build_vocab(testPrep)
testModel.train(testPrep, total_examples=testModel.corpus_count, epochs=testModel.iter)

<ipython-input-8-5d8c330931b2>:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  clemModel.train(clemPrep, total_examples=clemModel.corpus_count, epochs=clemModel.iter)
<ipython-input-8-5d8c330931b2>:10: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  lukeModel.train(lukePrep, total_examples=lukeModel.corpus_count, epochs=lukeModel.iter)
<ipython-input-8-5d8c330931b2>:14: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  pastModel.train(pastPrep, total_examples=pastModel.corpus_count, epochs=pastModel.iter)
<ipython-input-8-5d8c330931b2>:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  paulModel.train(paulPrep, total_examples=paulModel.corpus_count, epochs=paulModel.iter)
<ipython-input-8-5d8c330931b2>:22: DeprecationWarning: Call to de

(3194, 19290)

In [10]:
#returns tags found in all the texts
def getSimTags():
    simWords = []
    for word in testPrep:
        if (word in longPrep and word in markPrep and word in mathPrep and word in ignatPrep):
            simWords.append(word)
    return simWords

In [11]:
#transforms lst to list format
def convertToList(lst):
    ls = []
    for sentence in lst:
        for word in sentence:
            ls.append(word)
    return ls

In [16]:
#takes in a W2V model and returns the associated vocabulary of the model
def intoDict (model): 
    # Get the ordered list of words in the vocabulary
    words = model.wv.vocab.keys()
    # Make a dictionary
    we_dict = {word:model.wv[word] for word in words}
    return we_dict
#takes a dictionary "dct", a list of words "wordList", and optionally a label
#takes each word vector of the words in wordList and adds them to an overall vector, then returns this vector

def createVec(dct, wordList, label="NA"):
    vec = []
    for word in wordList:
        vec.append(dct[word])
    if (label != "NA"):
        vec.append(label)
    return vec

#returns all similar words of the texts
def getAllSimWords():
    simWords = []
    for word in testDict:
        puncuation = [",", "."]
        if (word in clemDict and word in lukeDict and word in pastDict and word in paulDict and word not in puncuation):
            simWords.append(word)
    return simWords

#creating feature vectors of the models
clemDict = intoDict(clemModel)
lukeDict = intoDict(lukeModel)
pastDict = intoDict(pastModel)
paulDict = intoDict(paulModel)
testDict = intoDict(testModel)

commonWordList = getAllSimWords()
#creating vectors to represent the models based on the commonWordList
clemVec = createVec(clemDict, commonWordList, 'Clement')
lukeVec = createVec(lukeDict, commonWordList, 'Luke')
pastVec = createVec(pastDict, commonWordList, 'Pastoral')
paulVec = createVec(paulDict, commonWordList, 'Paul')
testVec = createVec(testDict, commonWordList)

In [17]:
import numpy as np
#creating training and testing lists
trainX = np.array((clemVec[:len(commonWordList)], lukeVec[:len(commonWordList)], pastVec[:len(commonWordList)], paulVec[:len(commonWordList)]))
trainY =['Clement', 'Luke', "Pastoral", "Paul"]
testX = np.array(testVec[:len(commonWordList)])

In [25]:
#returns average results of MLPClassifier n (iterations) times
#although the variation for W2V was very small, I still wanted
# to make sure the results were as accurate as possible
def runIteration(iterations):
    avCl = 0
    avLu = 0
    avPa = 0
    avPu = 0
    for i in range(0, iterations):
        nsamples, nx, ny = trainX.shape
        trainXReshape = trainX.reshape((nsamples,nx*ny))
        from sklearn.neural_network import MLPClassifier
        clf = MLPClassifier(max_iter=8000, hidden_layer_sizes=(4,)).fit(trainXReshape, trainY)
        testXReshape = testX.reshape(1,testX.shape[1]*len(commonWordList))
        avCl += clf.predict_proba(testXReshape)[0][0]
        avLu += clf.predict_proba(testXReshape)[0][1]
        avPa += clf.predict_proba(testXReshape)[0][2]
        avPu += clf.predict_proba(testXReshape)[0][3]
    print("Average Clement: ", avCl/iterations)
    print("Average Luke: ", avLu/iterations)
    print("Average Pastoral: ", avPa/iterations)
    print("Average Paul: ", avPu/iterations)

In [26]:
runIteration(100)

Average Clement:  0.1210379484184341
Average Luke:  0.05472108666814164
Average Pastoral:  0.768235138207674
Average Paul:  0.05600582516389663
